In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,34173
2,Huelva,Confirmados PDIA 14 días,920
3,Huelva,Tasa PDIA 14 días,"179,2778221641951"
4,Huelva,Confirmados PDIA 7 días,496
5,Huelva,Total Confirmados,34359
6,Huelva,Curados,32401
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  34173.0
/tmp/ipykernel_73598/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9340.0
/tmp/ipykernel_73598/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_73598/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_73598/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_73598/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1240 personas en los últimos 7 días 

Un positivo PCR cada 587 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,34173.0,496.0,920.0,513170.0,96.654130,179.277822,250.0
Huelva-Costa,19759.0,326.0,615.0,289548.0,112.589277,212.400017,180.0
Huelva (capital),9340.0,116.0,245.0,143837.0,80.646843,170.331695,64.0
Condado-Campiña,11077.0,146.0,254.0,156231.0,93.451364,162.579770,62.0
Isla Cristina,2298.0,81.0,114.0,21393.0,378.628523,532.884588,46.0
Lepe,2230.0,33.0,59.0,27880.0,118.364419,211.621234,21.0
Niebla,280.0,31.0,31.0,4116.0,753.158406,753.158406,18.0
Ayamonte,1406.0,29.0,66.0,21104.0,137.414708,312.736922,18.0
Aljaraque,1202.0,18.0,33.0,21474.0,83.822297,153.674211,10.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Santa Bárbara de Casa,34.0,16.0,18.0,1043.0,1534.036433,1725.790988,5.0
Niebla,280.0,31.0,31.0,4116.0,753.158406,753.158406,18.0
Alosno,207.0,12.0,25.0,3933.0,305.110603,635.647089,6.0
Isla Cristina,2298.0,81.0,114.0,21393.0,378.628523,532.884588,46.0
Puerto Moral,5.0,1.0,1.0,278.0,359.712230,359.712230,NaN
Zarza-Perrunal (La),30.0,2.0,4.0,1253.0,159.616919,319.233839,0.0
Ayamonte,1406.0,29.0,66.0,21104.0,137.414708,312.736922,18.0
Lucena del Puerto,277.0,6.0,10.0,3261.0,183.992640,306.654400,2.0
Rosal de la Frontera,36.0,2.0,5.0,1697.0,117.855038,294.637596,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,579.0,1.0,4.0,7939.0,12.596045,50.384179,0.0,0.250000
Valverde del Camino,677.0,4.0,13.0,12750.0,31.372549,101.960784,0.0,0.307692
San Juan del Puerto,600.0,5.0,16.0,9411.0,53.129317,170.013814,3.0,0.312500
Manzanilla,110.0,1.0,3.0,2118.0,47.214353,141.643059,0.0,0.333333
Bollullos Par del Condado,1056.0,8.0,24.0,14387.0,55.605755,166.817266,3.0,0.333333
Bonares,314.0,5.0,15.0,6060.0,82.508251,247.524752,1.0,0.333333
Moguer,1444.0,16.0,44.0,21867.0,73.169616,201.216445,9.0,0.363636
Cartaya,1430.0,10.0,27.0,20083.0,49.793358,134.442065,5.0,0.370370
Rosal de la Frontera,36.0,2.0,5.0,1697.0,117.855038,294.637596,2.0,0.400000
